<a href="https://colab.research.google.com/github/gageswanston/BNFO301_Swanston_Gage/blob/main/BNFO301_Lab7_2_Student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Lab 7. Phylogenetic tree inference using distance matrix methods.

In this three part lab, you will infer a phylogenetic tree using a Distance Matrix Method.  This example uses one of a class of methods, which are statistically consistent, but sensitive to long-branch attraction.  In order to infer a tree, we must take several step, starting with sequences that have been aligned and are of equal length.  Next, we must find all pairwise distances and correct the distances using one of many models of subsitution.  Finally, we need to group sequence pairs and re-calculate distances, until all pairs have been grouped.  This grouping determines the order the tree, represented as a newick string.

##Part 1. 10 points - March 14

Problem 1.1 create a function to return P-distance

Problem 1.2 create a function to return corrected P-distance using the Jukes-Cantor model.

Problem 1.3 create a function to build a pairwise distance matrix

Problem 1.4 create a function to find the pair of sequences with smallest distance, given the distance matrix from part 1.3

##Part 2. 10 points - March 21

Problem 2.1 create a function to merge the most similar sequences based on the corrected P-distances and shrink the matrix

##Part 3. 10 points - March 28

Problem 3.1 create a function to infer a phylogenetic tree

Problem 3.2 create a function to read in a sequence alignment in phylip format

Problem 3.3 use functions created in parts 1 and 2 to infer a phylogenetic tree given the data provided in the phylip file

Problem 3.3 Answer questions about this tree

In [1]:
import math

Problem 1.1 Create function to calculate a P-distance given two sequences

In [2]:
#####################
#
#create a definition called "PDistance" that returns an uncorrected P-distance given two sequences
#your function should first check that sequences are of the same length, else return helpful error message (use keyword assert)
#recall that P-distance is the Hamming-distance / length of the alignment
#
#####################

def PDistance(x, y):
	assert len(x) == len(y), 'sequences must be of equal length'
	counter = 0.0
	for i in range(len(x)):
		if (x[i] != y[i]):
			counter += 1
	Pdist = counter/len(x)
	return Pdist

#####################
#
#use these two sequences to test your function;
#P-distance = 0.2
#
######################

seq1a = "TATAGTAAAA"
seq2a = "TATAGTATTA"
Dist_Problem1_1 = PDistance(seq1a, seq2a)
print(Dist_Problem1_1)

0.2


Problem 1.2. Create a function to correct the P-distance using the Jukes-Cantor model

In [3]:
#####################
#
#create a definition called "JCdistance" that returns a corrected P-distance given two sequences using the Jukes Cantor model
#recal, Jukes Cantor was described in class is a simple correction of P-distance
#Hint: use function created in problem 1.1, "PDistance", to simplfly your code
#
#####################

def JCdistance(x, y):
  uncorrectedDist = PDistance(x, y)
  if (uncorrectedDist != 0):
    correctdist = (-3/4)*math.log(1-(4/3)*uncorrectedDist)
  else:
    correctdist = 0.0
  return (correctdist)

#####################
#
#use these two sequences to test your function;
#Corrected P-distance = 0.2326161962278796
#
######################

seq1b = "TATAGTTAAA"
seq2b = "TATAGTTTTA"
JCdist_Problem1_2 = JCdistance(seq1b, seq2b)
print(JCdist_Problem1_2)

0.2326161962278796


Problem 1.3. Create a function to store distances between sequence pairs as a multidimentionsal dictionary.  You are required to use the Jukes Cantor corrected P-distance when building the dictionary.



In [4]:
#####################
#
#create a definition called "DistanceMatrix" that returns a dictionary given a set of DNA or AA sequences
#Input data: a dictionary containing sequences (values) and sequence names (keys)
#Output data: nested dictionary discribing pairwise distances between sequence pairs. Note: you should include all comparisons, including to self
#
#####################

def DistanceMatrix(sequences):
	matrix = {}
	for i in sequences.keys():
		if i not in matrix.keys():
			matrix[i] = {}
		for j in sequences.keys():
			matrix[i][j] = JCdistance(sequences[i], sequences[j])

	return matrix

#####################
#
#use these sequences to test your function
#expected result: {'seq1': {'seq1': 0.0, 'seq2': 0.2326161962278796, 'seq3': 0.3831192178244929, 'seq4': 0.2326161962278796}, 'seq2': {'seq1': 0.2326161962278796, 'seq2': 0.0, 'seq3': 0.3831192178244929, 'seq4': 0.0}, 'seq3': {'seq1': 0.3831192178244929, 'seq2': 0.3831192178244929, 'seq3': 0.0, 'seq4': 0.3831192178244929}, 'seq4': {'seq1': 0.2326161962278796, 'seq2': 0.0, 'seq3': 0.3831192178244929, 'seq4': 0.0}}
#
#####################

SampleSeqDict = {'seq1': 'TATACTAAAA', 'seq2': 'TATACTAATT', 'seq3': 'TATAGTATTA', 'seq4': 'TATACTAATT'}
Matrix_Problem1_3 = DistanceMatrix(SampleSeqDict)
print(Matrix_Problem1_3)

{'seq1': {'seq1': 0.0, 'seq2': 0.2326161962278796, 'seq3': 0.3831192178244929, 'seq4': 0.2326161962278796}, 'seq2': {'seq1': 0.2326161962278796, 'seq2': 0.0, 'seq3': 0.3831192178244929, 'seq4': 0.0}, 'seq3': {'seq1': 0.3831192178244929, 'seq2': 0.3831192178244929, 'seq3': 0.0, 'seq4': 0.3831192178244929}, 'seq4': {'seq1': 0.2326161962278796, 'seq2': 0.0, 'seq3': 0.3831192178244929, 'seq4': 0.0}}


Problem 1.4. Create a function to find the most similar sequences, given the nested dictionary you created in the last problem.

In [5]:
#####################
#
#create a definition called "GetMinDistance" that returns name of two sequences that has the shortest distance
#Note: avoid self comparisons when finding the shortest distance
#
#####################

def GetMinDistance(matrix):
	lowest_val = float("inf")
	lowest_i = ""
	lowest_j = ""

	for i in matrix.keys():
		for j in matrix.keys():
			if (matrix[i][j] < lowest_val) and (i != j):
				lowest_val = matrix[i][j]
				lowest_i = i
				lowest_j = j

	return lowest_i, lowest_j

#####################
#
#use these sequences to test your function, seq1 and seq2 are most similar within the test data
#Expected result: seq1 and seq2
#
#####################

SampleSeqDict = {'seq1': 'TATAGTATTA', 'seq2': 'TATAGTATTA', 'seq3': 'AATAGTATTA', 'seq4': 'TATACAATTA'}
Matrix_Problem1_4 = DistanceMatrix(SampleSeqDict)
i, j = GetMinDistance(Matrix_Problem1_4)
print(i,j)

seq1 seq2


Problem 2.1. Create a function to merge the closest neighbors and shrink the nested dictionary created in the previous meeting.

In [75]:
#####################
#
#create a definition called "ShrinkMatrix" that will reduce a nested dictionary
#
#steps to complete to reduce the nested dictionary:
#1. determine pair of labels to be grouped, based on similarity, use 'GetMinDistance' function created above
#2. determine new label for grouped pair - Newick format - (A,B)
#3. generate list of labels not being grouped, set of all labels - set of grouped pair
#4. populate a new nested dictionary with known distances from original nested dictionary
#5. add the new 'grouped taxa' to nested dictionary
#6. populate the nested dictionary with averaged distances
#
#####################
def ShrinkMatrix(matrix):

  N1, N2 = GetMinDistance(matrix)
  label="({0},{1})".format(N1,N2)
  namelist=list(set(matrix.keys()).difference(set([N1,N2])))
  nMatrix={}
  #print(namelist)
  for i in namelist:
    nMatrix[i]={}
    for j in namelist:
      nMatrix[i][j]=matrix[i][j]
  #print(nMatrix)
  nMatrix[label]={}

  for i in namelist:
    d1=matrix[i][N1]
    d2=matrix[i][N2]
    #print((d1+d2)/2)
    nMatrix[label][i]=((d1+d2)/2)

  return(nMatrix)











#####################
#
#use these sequences to test your function,
#Expected result: 'seq1' and 'seq2' should be grouped in the new dictionary
#
#####################

SeqDict_Problem2_1 = {'seq1': 'TATAGTATTA', 'seq2': 'TATAGTATTA', 'seq3': 'AATAGTATTA', 'seq4': 'TATACAATTA'}
Distances_Problem2_1 = DistanceMatrix(SeqDict_Problem2_1)
Matrix_Problem2_1 = ShrinkMatrix(Distances_Problem2_1)
print(Distances_Problem2_1)
print(Matrix_Problem2_1)

{'seq1': {'seq1': 0.0, 'seq2': 0.0, 'seq3': 0.10732563273050497, 'seq4': 0.2326161962278796}, 'seq2': {'seq1': 0.0, 'seq2': 0.0, 'seq3': 0.10732563273050497, 'seq4': 0.2326161962278796}, 'seq3': {'seq1': 0.10732563273050497, 'seq2': 0.10732563273050497, 'seq3': 0.0, 'seq4': 0.3831192178244929}, 'seq4': {'seq1': 0.2326161962278796, 'seq2': 0.2326161962278796, 'seq3': 0.3831192178244929, 'seq4': 0.0}}
{'seq4': {'seq4': 0.0, 'seq3': 0.3831192178244929}, 'seq3': {'seq4': 0.3831192178244929, 'seq3': 0.0}, '(seq1,seq2)': {'seq4': 0.2326161962278796, 'seq3': 0.10732563273050497}}
